# Global Country-wise Debt Analysis for the Past 5 Years

* 1: Define Scope
* Questions to address:
 - What are the global debt trends in the past 5 years?
 - How does the USA compare to other countries?
 - What potential risks and correlations exist between countries?



* 2. Import Required Libraries

In [130]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.preprocessing import MinMaxScaler


In [131]:


# 3: Data Collection
# Load the provided files (World Bank and IMF datasets)
world_bank_data = pd.read_csv('E:\DataAnalyst_Projects\Debt analysis\world_bank.csv')
weo_data = pd.read_csv('E:\DataAnalyst_Projects\Debt analysis\cleaned_weo_data.csv')


In [ ]:

# Inspect column names to verify correct formatting
print("World Bank Columns:", world_bank_data.columns)
print("IMF Columns:", weo_data.columns)

In [ ]:

# Display initial information about datasets
print("World Bank Data:")
print(world_bank_data.info())
print("\n IMF Data:")
print(weo_data.info())

In [134]:

# Step 4: Data Processing
# Harmonizing columns and identifying overlapping fields
world_bank_data.columns = world_bank_data.columns.str.lower().str.replace(' ', '_')
weo_data.columns = weo_data.columns.str.lower().str.replace(' ', '_')


In [ ]:
# Inspect column names to verify correct formatting
print("World Bank Columns:", world_bank_data.columns)
print("IMF Columns:", weo_data.columns)


In [136]:
# Reshape World Bank data if necessary
if 'year' not in world_bank_data.columns:
    years = [str(year) for year in range(1960, 2024 + 1)]  # List of years as strings
    valid_years = [year for year in years if year in world_bank_data.columns]
    world_bank_data = world_bank_data.melt(
        id_vars=['country', 'country_code', 'indicator_name', 'indicator_code', 'region', 'incomegroup'],
        value_vars=valid_years,
        var_name='year',
        value_name='debt'
    )
    world_bank_data['year'] = world_bank_data['year'].astype(int)

# Reshape IMF data if necessary
if 'year' not in weo_data.columns:
    years = [str(year) for year in range(2002, 2029 + 1)]  # List of years as strings
    valid_years = [year for year in years if year in weo_data.columns]
    weo_data = weo_data.melt(
        id_vars=['country', 'subject_descriptor'],
        value_vars=valid_years,
        var_name='year',
        value_name='debt'
    )
    weo_data['year'] = weo_data['year'].astype(int)


In [ ]:
print("World Bank Columns:", world_bank_data.columns)
print("IMF Columns:", weo_data.columns)

In [ ]:
# Verify if the 'year' column exists after reshaping
if 'year' not in world_bank_data.columns:
    raise KeyError("The 'year' column is still missing in the World Bank dataset after processing.")
if 'year' not in weo_data.columns:
    raise KeyError("The 'year' column is still missing in the IMF dataset after processing.")

# Filter data for the last 5 years
years_of_interest = [2020, 2021, 2022, 2023, 2024]
world_bank_filtered = world_bank_data[world_bank_data['year'].isin(years_of_interest)]
weo_filtered = weo_data[weo_data['year'].isin(years_of_interest)]

In [139]:

# Combine data on relevant keys (country, year, etc.)
combined_data = pd.merge(
    world_bank_filtered, 
    weo_filtered, 
    how='inner', 
    on=['country', 'year']
)

In [ ]:
# Exploratory Data Analysis (EDA)
print("EDA:")
print(combined_data.describe())

In [ ]:

# Identify missing values
missing_data = combined_data.isnull().sum()
print("Missing data:")
print(missing_data)



In [ ]:
print(world_bank_data.isnull().sum())



In [ ]:
print(weo_data.isnull().sum())

In [144]:
world_bank_data['debt'] = world_bank_data.groupby('country')['debt'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
world_bank_data['debt'].fillna(world_bank_data['debt'].mean(), inplace=True)


In [146]:
world_bank_data.dropna(subset=['region', 'incomegroup'], inplace=True)


In [ ]:
print(world_bank_data.isnull().sum())


In [ ]:
print(combined_data.columns)


In [149]:
combined_data.columns = combined_data.columns.str.strip()


In [ ]:
print(combined_data[['debt_x', 'debt_y']].head())


In [ ]:
print(combined_data.groupby('country')['debt_x'].mean().sort_values(ascending=False).head(5))


In [152]:
combined_data['debt_x'] = pd.to_numeric(combined_data['debt_x'], errors='coerce')
combined_data['debt_y'] = pd.to_numeric(combined_data['debt_y'], errors='coerce')


In [ ]:
combined_data['debt_x'].fillna(0, inplace=True)
combined_data['debt_y'].fillna(0, inplace=True)


In [ ]:
print(combined_data.groupby('country')['debt_x'].mean().sort_values(ascending=False).head(5))


In [ ]:
combined_data['debt'] = combined_data['debt_x'].fillna(combined_data['debt_y'])
print(combined_data.groupby('country')['debt'].mean().sort_values(ascending=False).head(5))


In [ ]:
# Step 6: Analysis
# (a) Descriptive Analysis
print("Top 5 countries with the highest average debt:")
print(combined_data.groupby('country')['debt'].mean().sort_values(ascending=False).head(5))

In [ ]:
# (c) USA-Specific Analysis
usa_data = combined_data[combined_data['country'] == 'United States']
print("USA Debt Data (Last 5 Years):")
print(usa_data)

# Compare USA's average debt to global averages
global_avg_debt = combined_data.groupby('year')['debt'].mean()
usa_avg_debt = usa_data.groupby('year')['debt'].mean()

comparison = pd.DataFrame({
    'Year': global_avg_debt.index,
    'Global Average Debt': global_avg_debt.values,
    'USA Average Debt': usa_avg_debt.values
})
print("USA vs Global Average Debt Comparison:")
print(comparison)

# Visualization: USA vs Global Average Debt
plt.figure(figsize=(10, 6))
plt.plot(comparison['Year'], comparison['Global Average Debt'], label='Global Average Debt', marker='o')
plt.plot(comparison['Year'], comparison['USA Average Debt'], label='USA Average Debt', marker='o', linestyle='--')
plt.title('USA vs Global Average Debt (Last 5 Years)')
plt.xlabel('Year')
plt.ylabel('Debt')
plt.legend()
plt.grid(True)
plt.show()


In [158]:

# (b) Network Analysis - Correlation between countries
corr_matrix = combined_data.pivot_table(index='year', columns='country', values='debt').corr()
G = nx.Graph()
for i in corr_matrix.index:
    for j in corr_matrix.columns:
        if i != j and corr_matrix.loc[i, j] > 0.7:  # Threshold for high correlation
            G.add_edge(i, j, weight=corr_matrix.loc[i, j])

* Visualizing the network


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

plt.figure(figsize=(20, 16))  # Increase figure size for better visualization

# Increase k for more space between nodes
pos = nx.spring_layout(G, k=0.8)  

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=1300, node_color='skyblue', alpha=0.85)

# Draw edges
nx.draw_networkx_edges(G, pos, width=1.5, alpha=0.5)

# Draw labels with bounding boxes and rotation
labels = nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold')

for _, label in labels.items():
    label.set_rotation(30)  # Rotate labels
    label.set_bbox(dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.3'))

plt.title("Network Analysis of Countries Based on Debt Correlation", fontsize=22)
plt.tight_layout()
plt.show()


In [ ]:

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Example: Create a sample DataFrame for countries and their debt (replace this with your actual data)
data = {'country': ['United States', 'China', 'Brazil', 'Germany', 'India'],
        'debt': [30, 40, 15, 35, 25]}  # Example debt data

combined_data = pd.DataFrame(data)

# Compute the correlation matrix (replace with your actual data correlation)
# For demonstration, we use a synthetic correlation matrix between countries
corr_matrix = pd.DataFrame(np.random.rand(5, 5), columns=combined_data['country'], index=combined_data['country'])

# Set the correlation threshold (lowering this will identify more relationships)
threshold = 0.5  # Adjust this value to find more relationships

# Create a new graph
G = nx.Graph()

# Add edges based on the correlation matrix, only if correlation is above the threshold
for i in corr_matrix.index:
    for j in corr_matrix.columns:
        if i != j and corr_matrix.loc[i, j] > threshold:  # Only add edge if correlation > threshold
            G.add_edge(i, j, weight=corr_matrix.loc[i, j])

# Remove isolated nodes (nodes with no edges)
isolated = list(nx.isolates(G))
G.remove_nodes_from(isolated)

# Check if there are any nodes left after filtering
if len(G.nodes) == 0:
    print("No edges found above the threshold. Try lowering it further.")
else:
    # Node size based on some attribute (e.g., mean debt for each country)
    node_size = combined_data.groupby('country')['debt'].mean()
    node_size = MinMaxScaler(feature_range=(300, 1000)).fit_transform(node_size.values.reshape(-1, 1)).flatten()

    # Color map adjustment
    node_color = ['red' if node == 'United States' else
                  'orange' if node == 'China' else
                  'green' if node == 'Brazil' else
                  'skyblue'
                  for node in G.nodes]

    # Visualization
    plt.figure(figsize=(20, 16))  # Increase figure size for better visualization
    pos = nx.spring_layout(G, k=0.8, seed=42)  # Set positions using spring layout (k controls spacing)

    # Draw nodes with customized sizes and colors
    nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_color, alpha=0.85)

    # Draw edges with customized width and transparency
    nx.draw_networkx_edges(G, pos, width=1.5, alpha=0.5)

    # Draw labels with customized font size and weight
    labels = nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold')

    # Adjust label appearance
    for _, label in labels.items():
        label.set_rotation(30)  # Rotate labels for better visibility
        label.set_bbox(dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.3'))  # Add background box for labels

    # Title and display
    plt.title(f"Network Analysis of Countries Based on Debt Correlation (Threshold: {threshold})", fontsize=22)
    plt.tight_layout()
    plt.show()



In [161]:
# Step 7: Visualization - PowerBI Dashboard (Guidance for export)
# Save combined data for PowerBI
combined_data.to_csv('debt_analysis_data.csv', index=False)

# Recommended Visuals for PowerBI:
# - Line chart: Global debt trends
# - Bar chart: Top 5 countries by debt per year
# - Heatmap: Correlation matrix of debt levels
# - Network diagram: Relationship of debt between countries

# Step 8: Documentation
# The final report and detailed insights should be documented in markdown or PDF format.
# Use visuals, descriptive analysis, and network analysis findings to provide actionable insights.
